In [1]:
print("OK")

OK


In [2]:
%pwd

'/Users/nihal/Downloads/End-End-MediRAG-GEN-AI/research'

In [12]:
import os
import langchain_core
print(dir(langchain_core))

['VERSION', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_api', '_import_utils', 'agents', 'caches', 'callbacks', 'chat_history', 'documents', 'embeddings', 'env', 'exceptions', 'globals', 'language_models', 'load', 'messages', 'output_parsers', 'outputs', 'prompt_values', 'prompts', 'rate_limiters', 'retrievers', 'runnables', 'stores', 'surface_langchain_beta_warnings', 'surface_langchain_deprecation_warnings', 'tools', 'tracers', 'utils', 'vectorstores', 'version']


In [13]:
os.chdir('/Users/nihal/Downloads/End-End-MediRAG-GEN-AI')

In [14]:
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter  

In [15]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)  # it extracts info using this and using glob="*.pdf" it only loads pdf files and later it gives docs

    documents = loader.load()

    return documents

In [16]:
# Check if Data directory exists, if not, use the correct path
import os
data_path = 'Data/' if os.path.exists('Data/') else '/Users/nihal/Downloads/End-End-MediRAG-GEN-AI/Data/'
extracted_data = load_pdf_file(data=data_path)


In [5]:
#extracted_data

In [8]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    for i, d in enumerate(text_chunks):
        meta = d.metadata or {}

        # doc name from source path (by default pypdfloader usually sets "source")
        src = meta.get("source", "")
        meta["doc_name"] = os.path.basename(src) if src else meta.get("doc_name", "document")

        # page is typically 0 based in loaders we convert to 1 based display
        if "page" in meta and isinstance(meta["page"], int):
            meta["page_display"] = meta["page"] + 1
        else:
            meta["page_display"] = None

        # stable chunk id (simplified version)
        meta["chunk_id"] = meta.get("chunk_id") or f"{meta.get('doc_name','doc')}_p{meta.get('page_display','?')}_c{i}"

        d.metadata = meta
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))


Length of Text Chunks 5860


In [8]:
#text_chunks

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [13]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

/var/folders/g5/54vbnb5j1gdc46kk4fr0tvmr0000gn/T/ipykernel_9552/4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [12]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))


Length 384


In [13]:
#query_result


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "medicalbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [16]:
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [17]:
from langchain_pinecone import PineconeVectorStore
# we embed each chunk and upsert the embeddings into our Pinecone Index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings, 
    index_name=index_name
)

In [18]:
#loading existing index

from langchain_pinecone import PineconeVectorStore
# we embed each chunk and upsert the embeddings into our Pinecone Index
docsearch = PineconeVectorStore.from_existing_index(
    embedding=embeddings, 
    index_name=index_name
)

In [19]:
docsearch

In [20]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [21]:
retrieved_docs = docsearch.similarity_search("What is diabetes?")

In [22]:
retrieved_docs

[Document(id='116955f8-e361-4dba-b050-6a208ac55ab6', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 542.0, 'page_label': '543', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='begin to fall. A person with diabetes mellitus either does\nnot make enough insulin, or makes insulin that does not\nwork properly. The result is blood sugar that remains\nhigh, a condition called hyperglycemia.\nDiabetes must be diagnosed as early as possible. If\nleft untreated, it can damage or cause failure of the eyes,\nkidneys, nerves, heart, blood vessels, and other body\norgans. Hypoglycemia, or low blood sugar, may also be\ndiscovered through blood sugar testing. Hypoglycemia is'),
 Document(id='f3844b42-7633-432d-a6f0-dd3d53639b29', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 542.0, 'page

In [31]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0.4, max_tokens=500)



In [24]:
import langchain
print(langchain.__version__)

1.2.4


In [32]:
from langchain_classic.chains import LLMChain


In [33]:
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assitant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        
    ]
)

In [34]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [37]:
response = rag_chain.invoke({"input": "What is diabetes?"})
print(response["answer"])

Diabetes is a chronic condition where the body either does not produce enough insulin or the insulin produced does not work effectively, leading to high blood sugar levels, a condition known as hyperglycemia. If left untreated, diabetes can cause serious damage to various organs, including the eyes, kidneys, nerves, heart, and blood vessels. Early diagnosis and management are crucial to prevent complications.


In [3]:
# Add to your imports section (after existing imports)
from langchain_classic.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

/Users/nihal/anaconda3/envs/llmapp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# HyDE Prompt - generates hypothetical medical document
hyde_prompt_template = """You are a medical expert. Write a detailed medical passage that would answer this question as it would appear in a medical textbook.

Question: {question}

Write a factual medical passage (3-4 sentences) using proper medical terminology:"""

hyde_prompt = PromptTemplate(
    input_variables=["question"],
    template=hyde_prompt_template
)

In [6]:
# Initialize LLM for hypothesis generation (higher temperature for diversity)
from langchain_openai import ChatOpenAI
hyde_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=300)

# Create HyDE chain
hyde_chain = hyde_prompt | hyde_llm | StrOutputParser()

In [22]:
# Test: Generate hypothesis for a sample question
test_question = "What are the symptoms of diabetes?"

print("Original Question:")
print(test_question)
print("\n" + "="*60)

# Generate hypothesis
hypothesis = hyde_chain.invoke({"question": test_question})

print("\nGenerated Hypothesis (HyDE):")
print(hypothesis)
print("="*60)

Original Question:
What are the symptoms of diabetes?


Generated Hypothesis (HyDE):
Diabetes mellitus is characterized by a spectrum of symptoms that can vary depending on the type and duration of the disease. Common clinical manifestations include polyuria (excessive urination), polydipsia (excessive thirst), and polyphagia (increased hunger), which arise due to hyperglycemia and its osmotic effects. Patients may also experience unexplained weight loss, fatigue, blurred vision, and delayed wound healing. In cases of diabetic ketoacidosis, particularly in Type 1 diabetes, additional symptoms such as abdominal pain, nausea, vomiting, and altered mental status may occur, necessitating urgent medical intervention.


In [25]:
# These cells from your trials.ipynb:

# 1. Load embeddings
embeddings = download_hugging_face_embeddings()

# 2. Connect to Pinecone
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    embedding=embeddings, 
    index_name="medicalbot"
)

# 3. Initialize LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4, max_tokens=300)

In [26]:
# Test question
question = "What causes chest pain?"

print(f"Question: {question}\n")
print("="*80)

# BASELINE: Retrieve using original question
print("\n🔵 BASELINE RETRIEVAL (Direct Query)")
print("-"*80)
baseline_docs = docsearch.similarity_search(question, k=3)

for i, doc in enumerate(baseline_docs, 1):
    page = doc.metadata.get("page_display", "?")
    print(f"\n[{i}] Page {page}:")
    print(doc.page_content[:200] + "...")

print("\n" + "="*80)

# HYDE: Generate hypothesis first, then retrieve
print("\n🟢 HYDE RETRIEVAL (Hypothetical Document)")
print("-"*80)

# Step 1: Generate hypothesis
hypothesis = hyde_chain.invoke({"question": question})
print(f"\nGenerated Hypothesis:\n{hypothesis}\n")
print("-"*80)

# Step 2: Retrieve using hypothesis
hyde_docs = docsearch.similarity_search(hypothesis, k=3)

for i, doc in enumerate(hyde_docs, 1):
    page = doc.metadata.get("page_display", "?")
    print(f"\n[{i}] Page {page}:")
    print(doc.page_content[:200] + "...")

print("\n" + "="*80)
print("\n📊 COMPARISON:")
baseline_pages = [doc.metadata.get("page_display") for doc in baseline_docs]
hyde_pages = [doc.metadata.get("page_display") for doc in hyde_docs]

print(f"Baseline pages: {baseline_pages}")
print(f"HyDE pages:     {hyde_pages}")
print(f"Pages overlap:  {set(baseline_pages) & set(hyde_pages)}")

Question: What causes chest pain?


🔵 BASELINE RETRIEVAL (Direct Query)
--------------------------------------------------------------------------------

[1] Page 209.0:
nary artery muscle spasm of insufficient duration or
intensity to cause an actual heart attack.
Causes and symptoms
Angina causes a pressing pain or sensation of heavi-
ness, usually in the chest area...

[2] Page 339.0:
the region of the aorta within the chest are called thoracic
aortic aneurysms. Aneurysms that occur in the part of the
aorta within the abdomen are called abdominal aortic
aneurysms.
Thoracic aortic a...

[3] Page 208.0:
Angina
Definition
Angina is pain, “discomfort,” or pressure localized
in the chest that is caused by an insufficient supply of
blood ( ischemia ) to the heart muscle. It is also some-
times characteri...


🟢 HYDE RETRIEVAL (Hypothetical Document)
--------------------------------------------------------------------------------

Generated Hypothesis:
Chest pain, or thoracic pain, can ari

In [27]:
def hyde_rag_pipeline(question: str, k: int = 5):
    """
    Complete HyDE RAG pipeline: Question → Hypothesis → Retrieve → Answer
    """
    print(f"❓ Question: {question}\n")
    
    # Step 1: Generate hypothesis
    print("🧠 Step 1: Generating hypothesis...")
    hypothesis = hyde_chain.invoke({"question": question})
    print(f"   Hypothesis: {hypothesis[:150]}...\n")
    
    # Step 2: Retrieve using hypothesis
    print(f"🔍 Step 2: Retrieving top-{k} documents using hypothesis...")
    retrieved_docs = docsearch.similarity_search(hypothesis, k=k)
    pages = [doc.metadata.get("page_display") for doc in retrieved_docs]
    print(f"   Retrieved pages: {pages}\n")
    
    # Step 3: Generate final answer
    print("💬 Step 3: Generating final answer...")
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    final_prompt = f"""You are a medical assistant. Based on the context below, answer the question concisely.

Context:
{context}

Question: {question}

Answer (2-3 sentences):"""
    
    answer_response = llm.invoke(final_prompt)
    answer = answer_response.content
    print(f"   Answer: {answer}\n")
    
    return {
        "question": question,
        "hypothesis": hypothesis,
        "retrieved_pages": pages,
        "answer": answer,
        "sources": retrieved_docs
    }

# Test the full pipeline
result = hyde_rag_pipeline("What are common symptoms of asthma?")

❓ Question: What are common symptoms of asthma?

🧠 Step 1: Generating hypothesis...
   Hypothesis: Asthma is a chronic inflammatory disorder of the airways characterized by recurrent episodes of wheezing, dyspnea (shortness of breath), chest tightne...

🔍 Step 2: Retrieving top-5 documents using hypothesis...
   Retrieved pages: [395.0, 394.0, 396.0, 394.0, 394.0]

💬 Step 3: Generating final answer...
   Answer: Common symptoms of asthma include wheezing, shortness of breath, coughing, and a feeling of tightness in the chest. In children, symptoms may also include itching on the back or neck. These symptoms can be triggered by factors such as cold air, exercise, stress, allergens, and respiratory infections.



In [28]:
# Test questions
test_questions = [
    "What is diabetes?",
    "What causes chest pain?",
    "What are symptoms of asthma?",
    "How is hypertension treated?",
    "What is appendicitis?"
]

comparison_results = []

for q in test_questions:
    print(f"\n{'='*80}")
    print(f"Testing: {q}")
    print(f"{'='*80}\n")
    
    # Baseline
    baseline_docs = docsearch.similarity_search(q, k=3)
    baseline_pages = [d.metadata.get("page_display") for d in baseline_docs]
    
    # HyDE
    hypothesis = hyde_chain.invoke({"question": q})
    hyde_docs = docsearch.similarity_search(hypothesis, k=3)
    hyde_pages = [d.metadata.get("page_display") for d in hyde_docs]
    
    # Compare
    overlap = len(set(baseline_pages) & set(hyde_pages))
    
    comparison_results.append({
        "question": q,
        "baseline_pages": baseline_pages,
        "hyde_pages": hyde_pages,
        "pages_overlap": overlap,
        "hypothesis": hypothesis[:100] + "..."
    })
    
    print(f"Baseline pages: {baseline_pages}")
    print(f"HyDE pages:     {hyde_pages}")
    print(f"Overlap:        {overlap}/3")
    print(f"Hypothesis:     {hypothesis[:150]}...")

# Show summary
print(f"\n{'='*80}")
print("SUMMARY")
print(f"{'='*80}\n")
avg_overlap = sum(r["pages_overlap"] for r in comparison_results) / len(comparison_results)
print(f"Average page overlap: {avg_overlap:.1f}/3")
print(f"Different retrieval in: {sum(1 for r in comparison_results if r['pages_overlap'] < 3)}/{len(comparison_results)} questions")


Testing: What is diabetes?

Baseline pages: [543.0, 199.0, 542.0]
HyDE pages:     [543.0, 416.0, 416.0]
Overlap:        1/3
Hypothesis:     Diabetes mellitus is a chronic metabolic disorder characterized by hyperglycemia, which results from defects in insulin secretion, insulin action, or ...

Testing: What causes chest pain?

Baseline pages: [209.0, 339.0, 208.0]
HyDE pages:     [209.0, 421.0, 367.0]
Overlap:        1/3
Hypothesis:     Chest pain, or thoracalgia, can arise from a multitude of etiologies, broadly categorized into cardiac, pulmonary, gastrointestinal, musculoskeletal, ...

Testing: What are symptoms of asthma?

Baseline pages: [394.0, 395.0, 250.0]
HyDE pages:     [395.0, 395.0, 636.0]
Overlap:        1/3
Hypothesis:     Asthma is a chronic inflammatory disorder of the airways characterized by variable airflow obstruction and bronchial hyperresponsiveness. Common sympt...

Testing: How is hypertension treated?

Baseline pages: [293.0, 411.0, 217.0]
HyDE pages:     [293

In [29]:
# Questions with known correct pages
evaluation_questions = [
    {"question": "What is diabetes?", "expected_pages": [542, 543]},
    {"question": "What are symptoms of asthma?", "expected_pages": [395]},
]

print("RETRIEVAL QUALITY COMPARISON\n")
print(f"{'Question':<40} | {'Method':<10} | {'Found':<6} | {'Pages'}")
print("-" * 80)

for item in evaluation_questions:
    q = item["question"]
    expected = set(item["expected_pages"])
    
    # Baseline
    baseline_docs = docsearch.similarity_search(q, k=5)
    baseline_pages = set([d.metadata.get("page_display") for d in baseline_docs if d.metadata.get("page_display")])
    baseline_hit = len(baseline_pages & expected) > 0
    
    # HyDE
    hypothesis = hyde_chain.invoke({"question": q})
    hyde_docs = docsearch.similarity_search(hypothesis, k=5)
    hyde_pages = set([d.metadata.get("page_display") for d in hyde_docs if d.metadata.get("page_display")])
    hyde_hit = len(hyde_pages & expected) > 0
    
    print(f"{q:<40} | {'Baseline':<10} | {'✓' if baseline_hit else '✗':<6} | {sorted(baseline_pages)}")
    print(f"{'':<40} | {'HyDE':<10} | {'✓' if hyde_hit else '✗':<6} | {sorted(hyde_pages)}")
    print(f"{'':<40} | {'Expected':<10} | {'':<6} | {sorted(expected)}")
    print("-" * 80)

RETRIEVAL QUALITY COMPARISON

Question                                 | Method     | Found  | Pages
--------------------------------------------------------------------------------
What is diabetes?                        | Baseline   | ✓      | [199.0, 275.0, 278.0, 542.0, 543.0]
                                         | HyDE       | ✓      | [275.0, 276.0, 416.0, 543.0]
                                         | Expected   |        | [542, 543]
--------------------------------------------------------------------------------
What are symptoms of asthma?             | Baseline   | ✓      | [250.0, 393.0, 394.0, 395.0]
                                         | HyDE       | ✓      | [393.0, 394.0, 395.0, 396.0]
                                         | Expected   |        | [395]
--------------------------------------------------------------------------------


In [2]:
from langchain_classic.memory import ConversationBufferWindowMemory
print("✅ Memory module available")

✅ Memory module available


In [3]:
from langchain_classic.memory import ConversationBufferWindowMemory
from langchain_classic.chains import ConversationalRetrievalChain

In [4]:
baseline_memory = ConversationBufferWindowMemory(
    k=5,  # Keep last 5 conversation turns
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

/var/folders/g5/54vbnb5j1gdc46kk4fr0tvmr0000gn/T/ipykernel_9552/1699082983.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  baseline_memory = ConversationBufferWindowMemory(


In [6]:
hyde_memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history", 
    return_messages=True,
    output_key="answer"
)

In [15]:
# These cells from your trials.ipynb:



# 2. Connect to Pinecone
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    embedding=embeddings, 
    index_name="medicalbot"
)

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4, max_tokens=300)

In [17]:
# Create retriever from your existing docsearch
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 5})

print("✅ Retriever created")

✅ Retriever created


In [18]:
baseline_conv_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=baseline_memory,
    return_source_documents=True,
    verbose=True  # Shows what's happening
)

In [19]:
print("="*60)
print("TESTING BASELINE RAG WITH MEMORY")
print("="*60)

# Question 1
print("\n📝 Question 1: What is diabetes?\n")
result1 = baseline_conv_chain.invoke({"question": "What is diabetes?"})
print(f"Answer: {result1['answer']}\n")

# Question 2 - Follow-up (tests if memory works)
print("\n📝 Question 2: What are its symptoms?\n")
result2 = baseline_conv_chain.invoke({"question": "What are its symptoms?"})
print(f"Answer: {result2['answer']}\n")

# Question 3 - Another follow-up
print("\n📝 Question 3: How is it treated?\n")
result3 = baseline_conv_chain.invoke({"question": "How is it treated?"})
print(f"Answer: {result3['answer']}\n")

print("="*60)
print("✅ If answers about diabetes symptoms/treatment, memory WORKS!")
print("="*60)

TESTING BASELINE RAG WITH MEMORY

📝 Question 1: What is diabetes?



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
begin to fall. A person with diabetes mellitus either does
not make enough insulin, or makes insulin that does not
work properly. The result is blood sugar that remains
high, a condition called hyperglycemia.
Diabetes must be diagnosed as early as possible. If
left untreated, it can damage or cause failure of the eyes,
kidneys, nerves, heart, blood vessels, and other body
organs. Hypoglycemia, or low blood sugar, may also be
discovered through blood sugar testing. Hypoglycemia is

Resources
BOOKS
Berkow, Robert, ed. The Merck Manual of Medical Informa-
tion: Home Edition. Whitehouse Station, NJ: Merck &
Co., Inc., 1997.
KEY TERMS
Aplas

In [20]:
print("\n🔍 MEMORY CONTENTS:")
print("="*60)
print(baseline_memory.load_memory_variables({}))
print("="*60)


🔍 MEMORY CONTENTS:
{'chat_history': [HumanMessage(content='What is diabetes?', additional_kwargs={}, response_metadata={}), AIMessage(content='Diabetes mellitus is a disorder of carbohydrate metabolism that occurs when a person either does not make enough insulin or makes insulin that does not work properly. This results in high blood sugar levels, a condition known as hyperglycemia. There are two main types of diabetes: type I, which is often referred to as juvenile onset or insulin-dependent diabetes, and type II. If left untreated, diabetes can lead to serious complications affecting various organs, including the eyes, kidneys, nerves, heart, and blood vessels.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='What are its symptoms?', additional_kwargs={}, response_metadata={}), AIMessage(content="The symptoms of diabetes mellitus include:\n\n- Fatigue\n- Abnormally high levels of glucose in the blood (hyperglycemia)\n- Incre

In [23]:
# For HyDE, we need custom integration since we generate hypothesis first

def hyde_rag_with_memory(question: str, memory: ConversationBufferWindowMemory):
    """
    HyDE RAG pipeline with conversation memory
    """
    # Step 1: Get conversation history
    history = memory.load_memory_variables({}).get("chat_history", [])
    
    # Step 2: Build context from history
    if history:
        history_text = "\n".join([
            f"{'User' if msg.type == 'human' else 'Assistant'}: {msg.content}"
            for msg in history
        ])
        question_with_context = f"{history_text}\n\nUser: {question}"
    else:
        question_with_context = question
    
    # Step 3: Generate HyDE hypothesis
    hypothesis = hyde_chain.invoke({"question": question_with_context})
    print(f"🧠 Hypothesis: {hypothesis[:150]}...\n")
    
    # Step 4: Retrieve using hypothesis
    retrieved_docs = docsearch.similarity_search(hypothesis, k=5)
    pages = [doc.metadata.get("page_display") for doc in retrieved_docs]
    print(f"📄 Retrieved pages: {pages}\n")
    
    # Step 5: Generate answer
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    final_prompt = f"""Based on this context, answer the question.

Context:
{context}

Question: {question}

Answer (2-3 sentences):"""
    
    answer_response = llm.invoke(final_prompt)
    answer = answer_response.content
    
    # Step 6: Save to memory
    memory.save_context(
        {"question": question},
        {"answer": answer}
    )
    
    return {
        "answer": answer,
        "hypothesis": hypothesis,
        "sources": retrieved_docs
    }

print("✅ HyDE with memory function created")

✅ HyDE with memory function created


In [25]:
# CREATE HYDE CHAIN (Add before Cell 6)


from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# HyDE prompt template
hyde_prompt_template = """You are a medical expert. Write a detailed medical passage that would answer this question as it would appear in a medical textbook.

Question: {question}

Write a factual medical passage (2-3 sentences) using proper medical terminology:"""

hyde_prompt = PromptTemplate(
    input_variables=["question"],
    template=hyde_prompt_template
)

# Create HyDE chain
hyde_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=200)
hyde_chain = hyde_prompt | hyde_llm | StrOutputParser()

print("✅ HyDE chain created")

✅ HyDE chain created


In [26]:
#testing HyDE with questions from med dataset
print("\n" + "="*60)
print("TESTING HYDE RAG WITH MEMORY")
print("="*60)

# Question 1
print("\n📝 Question 1: What is diabetes?\n")
result1 = hyde_rag_with_memory("What is diabetes?", hyde_memory)
print(f"Answer: {result1['answer']}\n")

# Question 2 - Follow-up
print("\n📝 Question 2: What are its symptoms?\n")
result2 = hyde_rag_with_memory("What are its symptoms?", hyde_memory)
print(f"Answer: {result2['answer']}\n")

# Question 3 - Another follow-up
print("\n📝 Question 3: How is it treated?\n")
result3 = hyde_rag_with_memory("How is it treated?", hyde_memory)
print(f"Answer: {result3['answer']}\n")

print("="*60)
print("✅ If answers about diabetes, HyDE memory WORKS!")
print("="*60)


TESTING HYDE RAG WITH MEMORY

📝 Question 1: What is diabetes?

🧠 Hypothesis: Diabetes mellitus is a chronic metabolic disorder characterized by hyperglycemia, resulting from defects in insulin secretion, insulin action, or both...

📄 Retrieved pages: [543.0, 276.0, 436.0, 199.0, 438.0]

Answer: Diabetes mellitus is a disorder of carbohydrate metabolism characterized by high blood sugar levels, known as hyperglycemia, due to either insufficient insulin production or ineffective use of insulin in the body. There are two main types: Type I diabetes, which results from a deficiency in insulin production, and Type II diabetes, which is primarily due to insulin resistance and can often be managed through lifestyle changes and medication.


📝 Question 2: What are its symptoms?

🧠 Hypothesis: Diabetes mellitus manifests with a constellation of symptoms primarily related to hyperglycemia, including polydipsia (excessive thirst), polyuria (in...

📄 Retrieved pages: [543.0, 438.0, 416.0, 544.0, 

In [27]:
#comparing both systems
# Reset memories
baseline_memory.clear()
hyde_memory.clear()

print("SIDE-BY-SIDE COMPARISON\n")
print("="*80)

test_conversation = [
    "What causes chest pain?",
    "What are the symptoms?",  # Follow-up
    "How is it diagnosed?"     # Follow-up
]

print("\n🔵 BASELINE + MEMORY:")
print("-"*80)
for q in test_conversation:
    result = baseline_conv_chain.invoke({"question": q})
    print(f"\nQ: {q}")
    print(f"A: {result['answer'][:200]}...")

print("\n" + "="*80)
print("\n🟢 HYDE + MEMORY:")
print("-"*80)
for q in test_conversation:
    result = hyde_rag_with_memory(q, hyde_memory)
    print(f"\nQ: {q}")
    print(f"A: {result['answer'][:200]}...")

print("\n" + "="*80)
print("✅ COMPARISON COMPLETE")
print("="*80)

SIDE-BY-SIDE COMPARISON


🔵 BASELINE + MEMORY:
--------------------------------------------------------------------------------


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
nary artery muscle spasm of insufficient duration or
intensity to cause an actual heart attack.
Causes and symptoms
Angina causes a pressing pain or sensation of heavi-
ness, usually in the chest area under the breast bone (ster-
num). It occasionally is experienced in the shoulder, arm,
neck, or jaw regions. Because episodes of angina occur
when the heart’s need for oxygen increases beyond the
oxygen available from the blood nourishing the heart, the

the region of the aorta within the chest are called thoracic
aortic aneurysms. Aneurysms that occur in the part of the
aorta 

In [28]:
#CELL 9: Test Memory Limit (k=5)
# Clear and test buffer limit
baseline_memory.clear()

print("TESTING MEMORY BUFFER LIMIT (k=5)\n")

# Ask 6 different questions
questions = [
    "What is diabetes?",
    "What is asthma?",
    "What is hypertension?",
    "What is pneumonia?",
    "What is appendicitis?",
    "What is acne?",
    "What was the first thing we discussed?"  # Should NOT remember diabetes
]

for i, q in enumerate(questions, 1):
    print(f"\n[{i}] Q: {q}")
    result = baseline_conv_chain.invoke({"question": q})
    print(f"A: {result['answer'][:150]}...")

print("\n" + "="*80)
print("✅ If last answer doesn't mention diabetes, buffer limit WORKS!")
print("="*80)

TESTING MEMORY BUFFER LIMIT (k=5)


[1] Q: What is diabetes?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
begin to fall. A person with diabetes mellitus either does
not make enough insulin, or makes insulin that does not
work properly. The result is blood sugar that remains
high, a condition called hyperglycemia.
Diabetes must be diagnosed as early as possible. If
left untreated, it can damage or cause failure of the eyes,
kidneys, nerves, heart, blood vessels, and other body
organs. Hypoglycemia, or low blood sugar, may also be
discovered through blood sugar testing. Hypoglycemia is

Resources
BOOKS
Berkow, Robert, ed. The Merck Manual of Medical Informa-
tion: Home Edition. Whitehouse Station, NJ: Merck &
Co., Inc., 1997.
KEY TERMS
Aplastic —E

In [29]:
# Clear both memories
baseline_memory.clear()
hyde_memory.clear()

print("🧹 Memory cleared for both systems")


🧹 Memory cleared for both systems
